In [1]:
import statsapi
from datetime import date

today = date.today().isoformat()

# Wrapper call for schedule
sched_data = statsapi.get("schedule", {"sportId": 1, "date": today})

games_today = []
for g in sched_data.get("dates", [])[0].get("games", []):
    games_today.append({
        "gamePk": g["gamePk"],
        "feed_link": f"https://statsapi.mlb.com{g['link']}"
    })

print("\nGames found today:")
for game in games_today:
    print(game)

# Pick the first game (or change index)
game_pk = games_today[0]["gamePk"]
feed_link = games_today[0]["feed_link"]

print("\nUsing gamePk:", game_pk)
print("Feed/live URL:", feed_link)



Games found today:
{'gamePk': 776413, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776413/feed/live'}
{'gamePk': 776409, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776409/feed/live'}
{'gamePk': 776405, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776405/feed/live'}
{'gamePk': 776412, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776412/feed/live'}
{'gamePk': 776406, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776406/feed/live'}
{'gamePk': 776411, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776411/feed/live'}
{'gamePk': 776407, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776407/feed/live'}
{'gamePk': 776400, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776400/feed/live'}
{'gamePk': 776399, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776399/feed/live'}
{'gamePk': 776410, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776410/feed/live'}
{'gamePk': 776408, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/gam

In [2]:
# Wrapper call for full game feed/live
feed_data = statsapi.get("game", {"gamePk": game_pk})

print("\nTop-level keys in feed/live response:")
print(list(feed_data.keys()))

# Inspect linescore + teams
linescore = feed_data.get("liveData", {}).get("linescore", {})
teams_section = feed_data.get("gameData", {}).get("teams", {})

print("\nLinescore keys:", list(linescore.keys()))
print("Team keys:", list(teams_section.keys()))  # ['home', 'away']



Top-level keys in feed/live response:
['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData']

Linescore keys: ['scheduledInnings', 'innings', 'teams', 'defense', 'offense']
Team keys: ['away', 'home']


In [3]:
import requests
import pandas as pd

feed_resp = requests.get(feed_link)   # wrapper doesn’t have feed/live
feed = feed_resp.json()

game_data = feed.get("gameData", {})
teams = game_data.get("teams", {})
status = game_data.get("status", {})

linescore = feed.get("liveData", {}).get("linescore", {})

row = {
    "gamePk": game_pk,
    "status": status.get("abstractGameState"),
    "inning": linescore.get("currentInning"),
    "inningHalf": linescore.get("inningHalf"),
    "outs": linescore.get("outs"),

    "home_team_id": teams.get("home", {}).get("id"),
    "home_team_name": teams.get("home", {}).get("name"),
    "home_score": linescore.get("teams", {}).get("home", {}).get("runs"),

    "away_team_id": teams.get("away", {}).get("id"),
    "away_team_name": teams.get("away", {}).get("name"),
    "away_score": linescore.get("teams", {}).get("away", {}).get("runs"),
}

df_scoreboard = pd.DataFrame([row])
df_scoreboard


,gamePk,status,inning,inningHalf,outs,home_team_id,home_team_name,home_score,away_team_id,away_team_name,away_score
0,776413,Preview,None,None,None,114,Cleveland Guardians,None,118,Kansas City Royals,None


In [4]:
score_rows = []

for g in games_today:
    gpk = g["gamePk"]
    flink = g["feed_link"]

    resp = requests.get(flink)
    if resp.status_code != 200:
        print("Skipping gamePk", gpk, "status", resp.status_code)
        continue

    feed = resp.json()

    game_data = feed.get("gameData", {})
    teams = game_data.get("teams", {})
    status = game_data.get("status", {})

    linescore = feed.get("liveData", {}).get("linescore", {})

    row = {
        "gamePk": gpk,
        "status": status.get("abstractGameState"),
        "inning": linescore.get("currentInning"),
        "inningHalf": linescore.get("inningHalf"),
        "outs": linescore.get("outs"),

        "home_team_id": teams.get("home", {}).get("id"),
        "home_team_name": teams.get("home", {}).get("name"),
        "home_score": linescore.get("teams", {}).get("home", {}).get("runs"),

        "away_team_id": teams.get("away", {}).get("id"),
        "away_team_name": teams.get("away", {}).get("name"),
        "away_score": linescore.get("teams", {}).get("away", {}).get("runs"),
    }
    score_rows.append(row)

df_scoreboard_all = pd.DataFrame(score_rows)
print("Shape:", df_scoreboard_all.shape)
df_scoreboard_all


Shape: (11, 11)


,gamePk,status,inning,inningHalf,outs,home_team_id,home_team_name,home_score,away_team_id,away_team_name,away_score
0,776413,Preview,None,None,None,114,Cleveland Guardians,None,118,Kansas City Royals,None
1,776409,Preview,None,None,None,146,Miami Marlins,None,120,Washington Nationals,None
2,776405,Preview,None,None,None,143,Philadelphia Phillies,None,121,New York Mets,None
3,776412,Preview,None,None,None,144,Atlanta Braves,None,112,Chicago Cubs,None
4,776406,Preview,None,None,None,140,Texas Rangers,None,158,Milwaukee Brewers,None
5,776411,Preview,None,None,None,108,Los Angeles Angels,None,142,Minnesota Twins,None
6,776407,Preview,None,None,None,135,San Diego Padres,None,113,Cincinnati Reds,None
7,776400,Preview,None,None,None,136,Seattle Mariners,None,138,St. Louis Cardinals,None
8,776399,Preview,None,None,None,137,San Francisco Giants,None,109,Arizona Diamondbacks,None
9,776410,Preview,None,None,None,133,Athletics,None,111,Boston Red Sox,None


In [5]:
import sqlite3

# Point to the root db file (outside MLB Api Endpoints)
db_path = r"C:\Users\sneal\Dev\mlb-data-pipeline\mlb_data.db"

conn = sqlite3.connect(db_path)

df_scoreboard_all.to_sql("game_silver", conn, if_exists="replace", index=False) # in Game notebook

conn.close()

print("✅ teams_silver written to root mlb_data.db")


✅ teams_silver written to root mlb_data.db
